In [1]:
from pprint import pprint

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="mistral", request_timeout=120.0)

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/home/ec2-user/miniconda3/envs/llindex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
!mkdir data/cookbook
!wget "https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1" -O data/cookbook/drake_kendrick_beef.pdf
!wget "https://www.dropbox.com/scl/fi/nts3n64s6kymner2jppd6/drake.pdf?rlkey=hksirpqwzlzqoejn55zemk6ld&st=mohyfyh4&dl=1" -O data/cookbook/drake.pdf
!wget "https://www.dropbox.com/scl/fi/8ax2vnoebhmy44bes2n1d/kendrick.pdf?rlkey=fhxvn94t5amdqcv9vshifd3hj&st=dxdtytn6&dl=1" -O data/cookbook/kendrick.pdf

mkdir: cannot create directory ‘data/cookbook’: File exists


--2024-06-26 20:31:07--  https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb8979cf1c56bbc4b220026cbbc.dl.dropboxusercontent.com/cd/0/inline/CVn3JtaoPLuj1u_ERKb7f-HLWteYwVj-VK2t7-gTgVVy4vrJJMdGakDdGC3bpxHE4A-r_1vrA8gTu3gqSdhn6QOHISdWmjEkpfA3ovlG1EmLk43u1VW7OXDvLtHnTDgPigI/file?dl=1# [following]
--2024-06-26 20:31:07--  https://ucb8979cf1c56bbc4b220026cbbc.dl.dropboxusercontent.com/cd/0/inline/CVn3JtaoPLuj1u_ERKb7f-HLWteYwVj-VK2t7-gTgVVy4vrJJMdGakDdGC3bpxHE4A-r_1vrA8gTu3gqSdhn6QOHISdWmjEkpfA3ovlG1EmLk43u1VW7OXDvLtHnTDgPigI/file?dl=1
Resolving ucb8979cf1c56bbc4b220026cbbc.dl.dropboxusercontent.com (ucb8979cf1c56bbc4b220026cbbc.dl.dropboxusercontent.com)... 162.125.6.15, 2620:100:6

In [7]:
from llama_index.core import SimpleDirectoryReader
docs_kendrick = SimpleDirectoryReader(input_files=["data/cookbook/kendrick.pdf"]).load_data()
docs_drake = SimpleDirectoryReader(input_files=["data/cookbook/drake.pdf"]).load_data()
docs_both = SimpleDirectoryReader(input_files=["data/cookbook/drake_kendrick_beef.pdf"]).load_data()


In [8]:
len(docs_kendrick), len(docs_drake), len(docs_both)

(69, 69, 34)

In [9]:
response = llm.complete("do you like drake or kendrick better?")

print(response)

 I don't have personal preferences. However, both Drake and Kendrick Lamar are incredibly talented artists who have made significant contributions to hip-hop music. Drake is known for his emotional storytelling and versatility, while Kendrick Lamar is renowned for his intelligent lyrics and socially conscious themes. It's a matter of personal taste and the specific aspects of their music that resonate with you.


In [10]:
stream_response = llm.stream_complete(
    "you're a drake fan. tell me why you like drake more than kendrick"
)

for t in stream_response:
    print(t.delta, end="")

 As a fan of both Drake and Kendrick Lamar, it's important to note that I appreciate the unique talents and contributions each artist brings to the music industry. However, when comparing the two, there are several reasons why I may find myself leaning towards Drake:

1. Emotional connection: Drake's music resonates deeply with me due to his ability to express complex emotions in a relatable way. His songs often deal with themes of love, heartbreak, friendship, and self-discovery, which I can easily relate to in my own life.

2. Storytelling abilities: Drake is a master storyteller who weaves intricate narratives throughout his albums, creating a cohesive listening experience that keeps me engaged from start to finish. His attention to detail and ability to craft compelling narratives are unparalleled in contemporary hip-hop.

3. Musical versatility: Drake's musical versatility is another factor that sets him apart. From soulful R&B ballads to upbeat party anthems, he effortlessly tran

In [11]:

from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Kendrick."),
    ChatMessage(role="user", content="Write a verse."),
]
response = llm.chat(messages)

In [12]:
print(response)

assistant:  (Verse)

I'm the voice of the streets, no longer the kid
Now I stand tall and speak truth to this system that's sick
Tired of the oppression, tired of the lies
Time for the revolution, open up your eyes

From Compton to the world, my words travel far
Every bar a testament, a soul bared
Struggling through the darkness, searching for the light
Fighting for the future, fighting for our right

We're more than our circumstances, we're more than the pain
Rising from the ashes, reclaiming our name
In the face of adversity, we stand together
United by the dream, united by the laughter

So hear me now, listen close
I'm just a voice, but I hope it helps you find your own
A beacon in the night, a guiding star
Let's change the world, let our love conquer all.


In [13]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs_both)
query_engine = index.as_query_engine(similarity_top_k=3)

In [14]:
response = query_engine.query("Tell me about family matters")

In [15]:
print(str(response))

 In the given context, "family matters" refers to a song by Drake titled "Family Matters," which is part of his response to Kendrick Lamar's diss track "The Heart Part 5." This song contains accusations against Kendrick Lamar regarding domestic abuse, infidelity, and allegations about the paternity of his children. Additionally, there are references to Kendrick Lamar's family members, including his parents and a supposedly unknown 11-year-old daughter. The song also addresses Drake's family, particularly his father Dennis Graham and his son Adonis. The accusations and counter-accusations in the song escalate the ongoing feud between these two prominent rappers.


In [16]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(docs_both)
summary_engine = summary_index.as_query_engine()

In [17]:
response = summary_engine.query(
    "Given your assessment of this article, who won the beef?"
)

In [18]:
pprint(str(response))

(' In terms of content and strategy, it seems that Drake has taken a more '
 "aggressive stance in this exchange, with direct accusations about Kendrick's "
 'personal life, including allegations of domestic abuse, estrangement from '
 'his family, and paternity issues regarding one of his children. Drake also '
 'references past tracks by Kendrick to strengthen his claims. On the other '
 'hand, Kendrick has been more cryptic in his response, using symbolism and '
 'indirect references in his music.\n'
 '\n'
 "However, it's essential to remember that this is a creative exchange within "
 'the hip-hop genre, and the perception of "winning" can be subjective. Both '
 'artists have large fanbases who might view their responses differently. '
 'Ultimately, the competition serves as an inspiration for both artists and '
 'contributes to the evolution of hip-hop culture.\n'
 '\n'
 "In terms of public opinion, it's challenging to predict who would be "
 'considered the winner, as it depends 

In [19]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [20]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False, verbose=True
)

response = query_engine.query(
    "Tell me about the song meet the grahams - why is it significant"
)

Selecting query engine 0: While the song 'Meet the Grahams' might be significant in a specific context, such as personal memories or cultural relevance, it is not typically summarized with factual details about its significance. Therefore, choice (1) is more relevant for searching for specific facts about the song..


In [21]:
pprint(str(response))

(' The song "Meet the Grahams" is a track by Kendrick Lamar, which is '
 'considered significant in the context of his ongoing feud with Drake. This '
 'song is seen as a strategic move that applies back-to-back pressure on '
 'Drake, suffocating any potential response from him and putting the ball in '
 "Drake's court to hit back. It was released as a follow-up to another track "
 'by Kendrick Lamar, which sparked this feud, and it is notable for its '
 'assertions about Drake being a "sex pest" who keeps bad company. The song\'s '
 'title, "Meet the Grahams," appears to be a reference to Drake\'s family or '
 'associates, and the cover art for the single depicts an image mocking up '
 "Drake's Toronto mansion as being on the sex offender registry. This track is "
 'significant in the ongoing Kendrick Lamar/Drake beef because it sets a '
 'challenging tone for their exchange of diss tracks.')


In [22]:
drake_index = VectorStoreIndex.from_documents(docs_drake)
drake_query_engine = drake_index.as_query_engine(similarity_top_k=3)

kendrick_index = VectorStoreIndex.from_documents(docs_kendrick)
kendrick_query_engine = kendrick_index.as_query_engine(similarity_top_k=3)

In [23]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

drake_tool = QueryEngineTool(
    drake_index.as_query_engine(),
    metadata=ToolMetadata(
        name="drake_search",
        description="Useful for searching over Drake's life.",
    ),
)

kendrick_tool = QueryEngineTool(
    kendrick_index.as_query_engine(),
    metadata=ToolMetadata(
        name="kendrick_summary",
        description="Useful for searching over Kendrick's life.",
    ),
)

In [24]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [drake_tool, kendrick_tool],
    llm=llm,  # llama3-70b
    verbose=True,
)

response = query_engine.query("Which albums did Drake release in his career? Answer as bullet points")

print(response)

Generated 1 sub questions.
[drake_search] Q: What albums has Drake released in his career
[drake_search] A:  In his career, Drake has released several albums including So Far Gone (2009), Thank Me Later (2010), Take Care (2011), Nothing Was The Same (2013), If You're Reading This It's Too Late (2015), Views (2016), More Life (2017), Scary Hours 2 (2021 EP), and Certified Lover Boy (2021). He has also released compilation albums like Care Package (2018) and Dark Lane Demo Tapes (2020).
 - So Far Gone (2009)
- Thank Me Later (2010)
- Take Care (2011)
- Nothing Was The Same (2013)
- If You're Reading This It's Too Late (2015)
- Views (2016)
- More Life (2017)
- Scary Hours 2 (2021 EP)
- Certified Lover Boy (2021)
- Care Package (2018 - Compilation Album)
- Dark Lane Demo Tapes (2020 - Compilation Album)


In [25]:
# !mkdir data/chinook
# !wget "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O "./data/chinook/chinook.zip"
# !unzip "./data/chinook/chinook.zip"

In [26]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column
)

engine = create_engine("sqlite:///data/chinook/chinook.db")

In [27]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [28]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    llm=llm,
)

In [29]:
response = query_engine.query("List 10 albums? Give answer as bullets")

print(response)

 - '...And Justice For All'
- '20th Century Masters - The Millennium Collection: The Best of Scorpions'
- 'A Copland Celebration, Vol. I'
- 'A Matter of Life and Death'
- 'A Real Dead One'
- 'A Real Live One'
- 'A Soprano Inspired'
- 'A TempestadeTempestade Ou O Livro Dos Dias'
- 'A-Sides'
- 'Ace Of Spades'


In [30]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/chinook/chinook.db')
df = pd.read_sql("select * from albums limit 5;", conn)
df

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [31]:
response = query_engine.query("What are some artists? Limit it to 5.")

print(response)

 Based on the query results, some artists are AC/DC, Accept, Aerosmith, Alanis Morissette, and Alice in Chains.


In [32]:
response = query_engine.query(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

print(response)


 Here are three tracks from the artist AC/DC: "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up". These tracks are all from the album "For Those About To Rock We Salute You".


In [33]:
print(response.metadata["sql_query"])


SELECT tracks.Name, albums.Title
FROM tracks
JOIN albums ON tracks.AlbumId = albums.AlbumId
JOIN artists ON albums.ArtistId = artists.ArtistId
WHERE artists.Name = 'AC/DC'
LIMIT 3;


In [34]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about the Kendrick and Drake beef."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [35]:
response = chat_engine.chat(
    "Tell me about the songs Drake released in the beef."
)
print(str(response))

Condensed question: Tell me about the songs Drake released in the beef.
Context: page_label: 31
file_path: data/cookbook/drake_kendrick_beef.pdf

Culture
Shaboo zey’s Cowboy Carter Features Were Only the Be ginning
By Heven Haile
Sign up for Manual, our new flagship newsletter
Useful advice on style, health, and more, four days a week.
5/10/24, 10:08 PM The Kendrick Lamar/Drake Beef, Explained | GQ
https://www.gq.com/story/the-kendrick-lamar-drake-beef-explained 31/34

page_label: 18
file_path: data/cookbook/drake_kendrick_beef.pdf

Kurrco
@Kurrco·Follow
KENDRICK LAMAR
6 16 IN LA
(DRAKE DISS)
OUT NOW 
This video has been deleted.
6 08 AM · May 3, 2024
59.3K Reply Copy link
Read 1.3K replies
After all this talk about “the clock,” who among us expected Kendrick to follow up his
own titanic diss track with another missile just three days later? Friday morning he
released “6:16 in LA,” with its title of course being a nod to Drake's series of time-stamp-
Sign up for Manual, our new flagshi

In [36]:
response = chat_engine.chat("What about Kendrick?")
print(str(response))

Condensed question:  "What was Kendrick Lamar's response during the beef with Drake?"
Context: page_label: 10
file_path: data/cookbook/drake_kendrick_beef.pdf

Dreamville Festival last weekend: “I don't care what Cole think, that Dot shit was weak
as fuck.”
Drake ends the original version of the song daring Kendrick to drop the nuclear diss he's
reportedly sitting on or “shut your mouth,” alleging that Kendrick has had one for “four
years” and still hasn't pushed the button. As direct as this Drake song is, the last few bars
imply it's just a move to get Kendrick to reply, at which point Drake has more
information ready to weaponize.
But the blink-and-you'll-miss-it reference to Kendrick's partner Whitney ("I'll be with
some bodyguards like Whitney") suggests Drake may not be considering the way his last
high profile beef turned out. In 2018 when he rapped to Pusha T that his crew would
“let it ring on him like Virginia Williams,” Pusha took the name-drop of his then fiancé
as an excus

In [40]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.agent import ReActAgent


from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [38]:
drake_tool = QueryEngineTool(
    drake_index.as_query_engine(),
    metadata=ToolMetadata(
        name="drake_search",
        description="Useful for searching over Drake's life.",
    ),
)

kendrick_tool = QueryEngineTool(
    kendrick_index.as_query_engine(),
    metadata=ToolMetadata(
        name="kendrick_search",
        description="Useful for searching over Kendrick's life.",
    ),
)

query_engine_tools = [drake_tool, kendrick_tool]

In [41]:
agent = ReActAgent.from_tools(
    query_engine_tools,  ## TODO: define query tools
    llm=llm,
    verbose=True,
)

In [42]:
response = agent.chat("Tell me about how Kendrick and Drake grew up")
print(str(response))

Thought: The current language of the user is not specified, I will assume English for this response. I need to use a tool to help me answer the question.
Action: kendrick_search
Action Input: {'input': 'Kendrick Lamar and Drake upbringing'}
Observation:  Kendrick Lamar was born in Compton, California to parents who were African Americans from the South Side of Chicago. He lived in Section 8 housing, was reliant on welfare and food stamps, and experienced homelessness during his childhood. Lamar grew up with close affiliates of the Westside Pirus gang, despite not being a member himself. His family lived in relative hardship, but he remembered having "good memories" from his childhood that sparked his interest in hip hop music.

On the other hand, Drake was born and raised in Toronto, Canada to African-Canadian parents. He grew up in a middle-class neighborhood and attended Jewish day school before switching to a public high school. Drake's father was a drummer, which influenced Drake's

In [49]:
response = agent.chat("Are Kendrick and Drake brothers?")
print(str(response))

Thought: (Implicit) I can answer without any more tools!
Answer:  No, Kendrick Lamar and Drake are not brothers. They are two separate individuals who grew up in different environments but have had success in the music industry.
 No, Kendrick Lamar and Drake are not brothers. They are two separate individuals who grew up in different environments but have had success in the music industry.


In [50]:
response = agent.chat("Who is older,  Kendrick or Drake?")
print(str(response))

Thought: (Implicit) I can answer without any more tools!
Answer:  To answer your question again, Drake was born on October 24, 1985, while Kendrick Lamar was born on June 17, 1987. Therefore, Drake is older by approximately 1 year and 4 months compared to Kendrick Lamar.
 To answer your question again, Drake was born on October 24, 1985, while Kendrick Lamar was born on June 17, 1987. Therefore, Drake is older by approximately 1 year and 4 months compared to Kendrick Lamar.


In [47]:
# https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook_ollama_replicate/